## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

1. children — количество детей в семье
2. days_employed — общий трудовой стаж в днях
3. dob_years — возраст клиента в годах
4. education — уровень образования клиента
5. education_id — идентификатор уровня образования
6. family_status — семейное положение
7. family_status_id — идентификатор семейного положения
8. gender — пол клиента
9. income_type — тип занятости
10. debt — имел ли задолженность по возврату кредитов
11. total_income — ежемесячный доход
12. purpose — цель получения кредита

## Что предстоить сделать?

1. определить и заполнить пропущенные значения:
2. заменить вещественный тип данных на целочисленный:
3. удалить дубликаты
4. выделить леммы в значениях столбца с целями получения кредита
5. категоризировать данные

In [2]:
import pandas as pd
df = pd.read_csv('***')
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
df.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

После того как мы ознакомились с таблицей можно сделать вывод, что:

1. В столбце "total_income" и  "days_employed" колличество строк гораздо меньше, чем в остальных. Это говорит о том, что в данном столбце есть какие-то проблемы с данными.
2.  Некорректно отображается минимальный и максимальный показатель по детям(ну 20 детей вполне вероятно, что такое возможно, но не будем упускать этот момент из виду, но -1 ребёнок...) 
3. Так же огромный разброс в колличестве стажа работы. Данный стоблец имеет отрцательные показатели так и сверхположительные. Максимальное значение 400 тыс. дней, что равно ~100 годам работы при максмальном занчении в столбце "dob_years" в 75 лет. Что явно говорит о том, что данные отображаются неккоректно.
4. Возраст. Минимальное значение равняется 0. Тут два варианта. Либо он не указан, либо какая-то ошибка. 
5. В столбце education значения имеют разный регистр, но одинаковый id.
6. в столце purpose имеются одинаковые по смыслу цели, но разные по описанию
7. столбец days_employed имеет тип float64, что не совсем корректно для наших исследований, т.к. нельзя сказать, что человек отработал 25.62737237 дней

### Шаг 2. Предобработка данных

### Обработка пропусков

Начнём с первого наблюдения. Взглянем есть ли пропуска в нашей таблице.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видим пропуска лежат в двух столбцах "days_employed" и "total_income". А так же значения одинаковы. Так что попробуем взглянуть на них лично.

In [6]:
df.total_income.tail(20)

21505     75439.993167
21506    145541.993320
21507     98180.279152
21508     72638.590915
21509     73029.059379
21510              NaN
21511    140068.472941
21512    147301.457769
21513    250986.142309
21514    355988.407188
21515    109486.327999
21516    322807.776603
21517    178059.553491
21518    153864.650328
21519    115949.039788
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, dtype: float64

Ага, а вот и наш пропуск. Вглянем лично на этого человека

In [7]:
df.loc[21510]

children                                  2
days_employed                           NaN
dob_years                                28
education                           среднее
education_id                              1
family_status               женат / замужем
family_status_id                          0
gender                                    F
income_type                       сотрудник
debt                                      0
total_income                            NaN
purpose             приобретение автомобиля
Name: 21510, dtype: object

Пустые значения в тех столбцах именно те, что нас интересуют. Выдвинем теорию о том, что эти два столбца взаимосвязаны. Взглянем на других людей, предварительно отфильтровав их.

In [8]:
df_empty_values = df.query('days_employed == "NaN" & total_income == "NaN"')
df_empty_values

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Как видим по нашему фильтру, что мы получили 2174 значения, ровно столько сколько у нас пропусков в обеих наших стоблцах. Теория подтвердилась. Попробуем найти логическую связь почему в данных стобцах имеются пропуски, т.к. судя по таблице занятость у всех разная. Первая мысль которая приходит это то, что человек нарочно не указал свой доход и стаж работы, но тогда скорее всего данные столбцы были бы не взаимосвязанны и скорее всего не была бы указана занятость, так что вполне вероятно, что это техническая ошибка. Просто удалить значения у нас не выйдет, т.к. это целых 10% значений от общей таблицы и нулевые значения сильно повлияют на наши дальнейшие расчёты. Для этого посмотрим какой вид занятости у нас присутсвует в таблице для того, чтобы расчитать для каждого вида занятости средние значения по дням и доходам.

In [9]:
df_empty_values.income_type.value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Так, вид занятости мы нашли, но прежде чем приступать к расчётам средних показателей по каждой группе надо посмотреть что не так со столбцом "days_employed". Посмотрим на колличество отрицательных значений в нашем списке, а так же по их минимальному и максимальному показателю.

In [10]:
df.days_employed.loc[df['days_employed'] < 0].min() #т.к. имеем дело с отрицательными числами,
                                                    #то данный показатель будет показывать максимальное значение

-18388.949900568383

In [11]:
df.days_employed.loc[df['days_employed'] < 0].max() #т.к. имеем дело с отрицательными числами, 
                                                    #то данный показатель будет показывать минимальное значение

-24.14163324048118

In [12]:
df.days_employed.loc[df['days_employed'] < 0].median()

-1630.0193809778216

Как видим, что практически 2/3 стоблца имеют отрицательные показатели, а так же минимальный опыт работы похож на реальный показатель, так же как и максмальное значение не уходит в запредельные числа, так что это говорит о том что вряд ли это ошибка и скорее всего обратный отсчёт от даты выгрузки. Так же можем заметить, что медиана равна ~ 4,5 года, что говорит о том, что стаж работы указан на одном(действительном рабочем месте) отсюда можно и связать, что отрицательные значения возникают от даты выгрузки датасета. Т.е. при заполнения формы человек указал, что работает по настоящее время. Для того, чтобы подвердить нашу теорию обратимся к человеку с минимальным опытом работы

In [13]:
empl = df.query('days_employed == -24.14163324048118')
empl

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,-24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование


Обратим внимание, что данному человеку на данный момент 31 год и имеет среднее образование, так же что есть ребёнок и это женский пол. Возможно, человек не работал всю свою жизнь и была в декрете/домохозяйкой, но вспомним значение медианы и оставим нашу мысль на потом для других исследований.

Теперь посмотрим положительные показатели по отработанным дням и так же минимальное и максимальное значение

In [14]:
df.days_employed.loc[df['days_employed'] > 0].count()

3445

In [15]:
df.days_employed.loc[df['days_employed'] > 0].min()

328728.72060451825

In [16]:
df.days_employed.loc[df['days_employed'] > 0].max()

401755.40047533

In [17]:
df.days_employed.loc[df['days_employed'] > 0].median()

365213.30626573117

не так уж и много. По сравнению с отрицательными значениями. Но минимальные и максимальные значения уходят в заоблачные показатели, так же как и медиана. Попробуем найти взаимосвязь среди вида занятости.

In [18]:
positive_days_employed = df.query('days_employed > 0').income_type.value_counts()
positive_days_employed

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [19]:
negative_days_employed = df.query('days_employed < 0').income_type.value_counts()
negative_days_employed

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
в декрете              1
студент                1
Name: income_type, dtype: int64

Отрицательные показатели отображаются у тех, кто на данный момент имеет трудовую занятость, либо так же имеет, но находится в отпуске(декрет), так же как и студент может иметь какую-либо трудовую деятельность(подработку). т.е. простыми словами, days_employed отображает сколько человек работает на данный момент на данной должности, т.е. это показатель не общего стажа работы, а только стажа его нынешней работы, отсюда же и следует вывод, что:
    1. положительные значения ошибочны и находятся в значениях у тех, кто сейчас не работает на данный момент, следовательно, все данные можно приравнять к нулю для корректных отображений наших данных.
Для этого возьмём пенсионеров и безработных и укажем, что их стаж равен 0.

In [20]:
df.loc[df['days_employed'] > 0, 'days_employed'] = 0
df.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь после того как мы привели наши данные к более реалистичным показателям, то теперь можно приступить к обработке значений NaN. Для этого воспользуемся расчётом медианы для каждого типа занятости, но для начала приведём наши отрицательные числа к положительным.

In [21]:
df.days_employed = df.days_employed.abs()
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))
df.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Отлично! После того, как мы наконец-то разобрались со стажем, то можно переходить уже к расчётам медианы достатка для каждого типа занятости. 

In [22]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))
df.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь посмотрим удалось ли нам заполнить наши пропуски.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Да. Замена прошла успешно, но прежде чем переходить к следущим расчётам взглянем на наши показатели.

In [24]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,1874.494481,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,2175.831518,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,316.240646,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1355.683356,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,2570.047544,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Отлично. Теперь стаж работы выглядит адекватно, но у нас так же осталась проблема с детьми. Так что приступим к этому шагу и посмотрим где у нас тут отклонения.

In [25]:
df.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как мы видим, что с 20 детьми аж целых 76 человек, что говорит о том, что это явно не уникальный случай, а какая-то ошибка, либо опечатка. Поэтому вглянем на этих людей и решим на какую цифру была опечатка. на 2 или на 0.
Так же видим, что у 47 людей -1 ребёнок, что в реальности невозможно. Тут возникает опять же вопрос. Опечатка или же какая-то ошибка? Поэтому так же взглянем на этих людей и возьмём за основу их семейное положение.

In [26]:
df.loc[df['children'] == 20].family_status.value_counts()

женат / замужем          49
гражданский брак         12
Не женат / не замужем     9
вдовец / вдова            4
в разводе                 2
Name: family_status, dtype: int64

Как мы можем видеть, что больше всех у женатых, либо же у тех кто в гражданском браке, что говорит о том, что скорее всего была опечатка на 0, а не на 2. Поэтому для этой группы приравняем колличество детей к 2, а не к 0.
Теперь перейдём к следующей группе и посмотрим на неё. Так же за наш показатель возьмём их семейное положение.

In [27]:
df.loc[df['children'] == -1].family_status.value_counts()

женат / замужем          29
гражданский брак          5
Не женат / не замужем     5
вдовец / вдова            4
в разводе                 4
Name: family_status, dtype: int64

Тут, практически, такая же картина. Так что приравняем наш показатель к положительному числу.

In [28]:
df.children = df.children.abs()
df.loc[df['children'] == 20, 'children'] = 2
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,1874.494481,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,0.755528,2175.831518,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,316.240646,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1355.683356,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,2570.047544,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Отлично. Этот вопрос решён и теперь всё достаточно понятно с детьми и показатели лежат в пределах реального. 
Теперь перейдём к следующему вопоросу и посмотрим, что не так у нас возрастом. Откуда взялся минимальный показатель 0 и как часто он всречается в нашей выборке. Но раз у нас есть нулевые значения, то возможно, есть и другие неверные показатели. Давайте вглянем на них.

In [29]:
df.query('dob_years < 17').dob_years.value_counts()

0    101
Name: dob_years, dtype: int64

Как видим, что у нас всего 1 единственный неверный показатель, что говорит о том, что человек скорее всего не указал свой возраст, либо случайно пропустил при заполнении формы. Поэтому заменим нулевые значения на медианные значения по каждому виду занятости.

In [30]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df.groupby('income_type')['dob_years'].transform('median')
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,1874.494481,43.496167,0.817236,0.972544,0.080883,1.652253e+05
std,0.755528,2175.831518,12.231538,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,316.240646,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1355.683356,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,2570.047544,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Отлично. Замена прошла успешно и наши нулевые показатели приняли медианные значения для каждой группы, но как мы видим, что колонка dob_years приняла вещественный тип, что не совсем корректно. Поэтому в следущем шаге преобразуем колонки days_employed и dob_years из вещественного к целочисленному типу данных.

In [32]:
df.query('income_type == "пенсионер"').agg({'dob_years':'median'})

dob_years    60.0
dtype: float64

### Вывод

Итак подведём итоги:

1. В столбце "total_income" и  "days_employed" заполнили пропуски и привели показатели к реальным значениям. Теперь нет космических показателей в столбце "days_employed".
2.  в столбце children заменили неверные данные на более действительные.
3. Заполнили нулевые значения в столбце "dob_years". Теперь у нас нет людей с возрастом 0 лет.

Что осталось сделать? 
1. В столбцах  "days_employed" и "dob_years" заменить вещественный тип данных на целочисленный, т.к. нельзя сказать, что человек отработал 2,232 дня и ему 32.232323 лет.
2. В столбце "education" ознакомиться с дубликатами и удалить их.
3. В столбце "purpose" процесс провести лемматизацию. 

### Замена типа данных

Итак, приступим к следущему шагу и попробуем преобразовать наши столбцы "days_employed" и "dob_years" к целочисленному типу данных. Для этого воспользуемся методом "astype()".

In [33]:
df['days_employed'] = df['days_employed'].astype(int)
df['dob_years'] = df['dob_years'].astype(int)
df[['days_employed','dob_years']]

,days_employed,dob_years
0,8437,42
1,4024,36
2,5623,33
3,4124,32
4,0,53
...,...,...
21520,4529,43
21521,0,67
21522,2113,38
21523,3112,38


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Теперь все данные представлены в удобной для чтения формате. Теперь можно переходить к следущему шагу.

### Обработка дубликатов

Итак, как мы могли заметить раннее, что некоторые значения в стоблце education имеют разный регистр. Ознакомимся сколько хранится таких значений в нашем датафрейме и после преобразования приступим к поиску дубликатов.

In [35]:
df.education.value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Как мы видим, что всего у нас представлено 5 видов образования, а в датафрейме отображается куда больше. Для начала приведём все их к нижнему регистру.

In [36]:
df['education'] = df.education.str.lower()
df.education.value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [37]:
df.groupby('education')['education_id'].value_counts()

education            education_id
высшее               0                5260
начальное            3                 282
неоконченное высшее  2                 744
среднее              1               15233
ученая степень       4                   6
Name: education_id, dtype: int64

Отлично. теперь всё встало на свои места. Можно приступить к поиску дубликатов.

In [38]:
df.duplicated().value_counts()

False    21454
True        71
dtype: int64

Итак, наш запрос нашёл 71 дубликат. Посмотрим на них.

In [39]:
df[df.duplicated()].sort_values(by='dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,1574,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,сыграть свадьбу
19321,0,1574,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594.396847,сделка с подержанным автомобилем
18328,0,1574,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594.396847,покупка жилой недвижимости
21281,1,1574,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,покупка коммерческой недвижимости
6312,0,1574,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
17338,0,0,64,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
20187,0,0,65,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
9528,0,0,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514.486412,операции со своей недвижимостью
9604,0,0,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,на проведение свадьбы


Как видим, это действительно дубликаты, так что просто удаляем их.

In [40]:
df = df.drop_duplicates()
df.duplicated().value_counts()

False    21454
dtype: int64

### Вывод

Дубликаты были найдены и удалены, т.к. в дальнейщем могут немного исказить наши расчёты.

### Лемматизация

Для дальнейшей категоризации данных воспользуемся лемматизацией слов в столбце "purpose". Для этого импортируем нужную нам библиотеку, но сначала взглянем на этот столбец. Иморт библиотеки запишем в начало, там где хранятся остальные библиотеки.

In [41]:
df.purpose.value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Как видим, что очень много одинаковых по смыслу слов, но написаны они по-разному. Для того, чтобы в дальнейшем категоризировать их - лемматизируем их и взглянем ещё раз.

In [42]:
df['category_purpose'] = df.purpose.apply(m.lemmatize)

In [43]:
df.category_purpose.value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

Изучив список, можно увидеть, что в каждой строке встречаются такие слова как:
    1. автомобиль
    2. свадьба
    3. образование
    4. жильё
    5. недвижимость
Но т.к. жильё относится к недвижимости, то можно объединить их. 
Для этого напишем функцию, которая будет искать в строчках знакомые нам слова и заменять их на одно слово.

In [44]:
def change(column):
    for row in column:
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'
        if 'образование' in row:
            return 'образование'
        if 'жилье' in row:
            return 'недвижимость'
        if 'недвижимость' in row:
            return 'недвижимость'        

После того как мы написали нашу функцию, то применим её к столбцу category_purpose

In [45]:
df.category_purpose = df.category_purpose.apply(change)

Теперь ознакомимся с таблицей и посмотрим не допустили ли мы где-то ошибку и все ли значения приняли нужную нам форму.

In [46]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


In [47]:
df.category_purpose.value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: category_purpose, dtype: int64

In [48]:
df.category_purpose.isnull().sum()

0

### Вывод

после лемматизации мы более точно сформулировали цели приобритения кредита для того, чтобы в дальнейшем можно было более точно ответить на наши вопросы. После предобработки наших данных приступим к категоризации.

### Категоризация данных

Теперь для того, чтобы ответить на поставленные вопросы, мы можем приступить к категоризации данных. Исходя из нашей таблицы мы можем разбить людей по:
    1. доходам
    2. наличию детей
    3. задолженности

Разобьём сначала людей по доходам. За основу возьмём данные Росстата, где сказано, что:
    1. бедные - < 17 000
    2. среднедостаточные - от 17 000 до 50 000
    3. состоятельные - от 50000 до 100 000
    4. богатые - от 100 000 до 500 000
    5. сверхбогатые - свыше 500 000
    
Для этого напишем функцию:

In [49]:
def name_social_class(row):
    income = row['total_income']
    if income < 17000:
        return 'бедные'
    if 17000 <= income <= 50000:
        return 'среднедостаточные'
    if 50001 <= income <= 100000:
        return 'состоятельные'
    if 100001 <= income <= 500000:
        return 'богатые'
    if income > 500000:
        return 'бедные'
    
df['social_status'] = df.apply(name_social_class, axis = 1)
df['social_status'].value_counts()

богатые              16769
состоятельные         4091
среднедостаточные      372
бедные                 222
Name: social_status, dtype: int64

Теперь разобьём людей на группу по колличеству детей. В России считается что:
    1. 1-2 ребёнка - традиционная семья
    2. больше 2 детей - многодетная семья
    3. нет детей - бездетная семья
    
Напишем функцию, которая определит какой гражданин относится к какой категории.

In [50]:
def count_kids(row):
    kid = row['children']
    if kid < 1:
        return 'нет детей'
    if kid >= 1:
        return 'есть дети'


df['type_of_family'] = df.apply(count_kids, axis = 1)
df['type_of_family'].value_counts()

нет детей    14091
есть дети     7363
Name: type_of_family, dtype: int64

In [51]:
def debts(row):
    deb = row['debt']
    if deb == 1:
        return 'должен'
    if deb == 0:
        return 'не должен'

df['actual_debt'] = df.apply(debts, axis = 1)
df['actual_debt'].value_counts()

не должен    19713
должен        1741
Name: actual_debt, dtype: int64

### Вывод

В данном блоке мы завершили финальную подготовку данных, а именно категоризацию данных, и готовы перейти к ответам на вопрос банка.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для начала взглянем на объём нашей выборки, чтобы в дальнейшем посмотреть на соотношение задолженности между людьми у которых есть дети и у которых их нет.

In [52]:
df.pivot_table('debt', ['type_of_family', 'actual_debt'], aggfunc = 'count')

debt
type_of_family actual_debt       
есть дети      должен         678
               не должен     6685
нет детей      должен        1063
               не должен    13028

In [61]:
#df.pivot_table(index='type_of_family', values='debt', aggfunc=['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
type_of_family,,,
есть дети,7363,678,0.092082
нет детей,14091,1063,0.075438


Как мы видим, что разница между коллчеством людей с детьми и без практически в 2 раза, но в целом объём выборки устраивает, чтобы провести дальнейшие расчёты. Рассчитаем соотношение.

In [53]:
df.groupby('type_of_family', as_index=False)\
    .agg({'debt':'mean'})\
    .sort_values(by='debt', ascending=False)\

,type_of_family,debt
0,есть дети,0.092082
1,нет детей,0.075438


### Вывод

Как видим, что те люди у которых нет детей более ответственны к своим долговым обязательствам, чем те, у которых они есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для начала взглянем на объём нашей выборки, чтобы в дальнейшем посмотреть на соотношение задолженности между людьми и их семейным положением

In [54]:
df.pivot_table('debt', ['family_status', 'actual_debt'], aggfunc = 'count')

debt
family_status         actual_debt       
Не женат / не замужем должен         274
                      не должен     2536
в разводе             должен          85
                      не должен     1110
вдовец / вдова        должен          63
                      не должен      896
гражданский брак      должен         388
                      не должен     3763
женат / замужем       должен         931
                      не должен    11408

Ознакомившись с талбицей можно рассчитать соотношение.

In [55]:
df.groupby('family_status', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,family_status,debt
0,Не женат / не замужем,0.097509
3,гражданский брак,0.093471
4,женат / замужем,0.075452
1,в разводе,0.071130
2,вдовец / вдова,0.065693


### Вывод

Как видим, что да. Чаще те люди, кто не имеет законных отношений более склонны к задолжности.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Так же как и в предыдущих шагах ознакомимся сначала с колличеством наших заёмщиков.

In [56]:
df.pivot_table('debt', ['social_status', 'actual_debt'], aggfunc = 'count')

debt
social_status     actual_debt       
бедные            должен          14
                  не должен      208
богатые           должен        1373
                  не должен    15396
состоятельные     должен         331
                  не должен     3760
среднедостаточные должен          23
                  не должен      349

Ознакомившись с талбицей можно рассчитать соотношение.

In [57]:
df.groupby('social_status', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,social_status,debt
1,богатые,0.081877
2,состоятельные,0.080909
0,бедные,0.063063
3,среднедостаточные,0.061828


### Вывод

Как видим, что больше всего в нашей выборке богатых людей, но так же, что у них самый высокий показатель задолженности. 

- Как разные цели кредита влияют на его возврат в срок?

Так же как и в предыдущих шагах ознакомимся сначала с колличеством наших заёмщиков.

In [58]:
df.pivot_table('debt', ['category_purpose', 'actual_debt'], aggfunc = 'count')

debt
category_purpose actual_debt       
автомобиль       должен         403
                 не должен     3903
недвижимость     должен         782
                 не должен    10029
образование      должен         370
                 не должен     3643
свадьба          должен         186
                 не должен     2138

Ознакомившись с талбицей можно рассчитать соотношение.

In [59]:
df.groupby('category_purpose', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,category_purpose,debt
0,автомобиль,0.093590
2,образование,0.092200
3,свадьба,0.080034
1,недвижимость,0.072334


### Вывод

Исходя из полученных данных можно увидеть, что самыми надежными заёмщиками являются те люди, чья цель кредита является в приобретении недвижимости и самые ненадёжные те, кто берёт кредит с целью свадьбы и образования.

### Шаг 4. Общий вывод


##### 1.  Те люди у которых нет детей более ответственны к своим долговым обязательствам, чем те, у которых они есть.
##### 2.  Те люди, кто не имеет законных отношений более склонны к задолжности.
##### 3. Чем ниже достаток, тем более ответственен заёмщик
##### 4.  те люди, чья цель кредита является в приобретении недвижимости являются самыми надёжными, чем те чья цель является свадьба или же автомобиль









## Заключение
Получив ответы на вопрос, мы можем сделать вывод, что самые надежные заёмщики те, кто:
    1. не имеет детей, его достаток составляет не выше среднего, в законных отношениях, либо имел их и цель их покупки - недвижимость

Соответственно, можно сделать вывод, что самые ненадежные заёмщики те, кто:
    1. имеет детей, имеет хороший достаток, не имеет законных отношений или не был в них, цель покупки - образование или автомобиль.
    